In [1]:
import pandas as pd
import numpy as np
import csv
import pickle
from sklearn import metrics
from sklearn.tree import DecisionTreeClassifier
from sklearn.model_selection import train_test_split, GridSearchCV

In [2]:
data = pd.read_csv("clean_loan.csv")
data.columns = data.columns.str.lower()

In [3]:
data

,loan_amnt,funded_amnt,revol_bal,policy_code,funded_amnt_inv,int_rate,installment,annual_inc,dti,delinq_2yrs,...,term,grade,emp_length,home_ownership,verification_status,loan_status,pymnt_plan,purpose,initial_list_status,application_type
0,2500,2500,4341,1,2500.0,13.56,84.92,55000.0,18.24,0.0,...,36 months,C,10+ years,RENT,Not Verified,Current,n,debt_consolidation,w,Individual
1,30000,30000,12315,1,30000.0,18.94,777.23,90000.0,26.52,0.0,...,60 months,D,10+ years,MORTGAGE,Source Verified,Current,n,debt_consolidation,w,Individual
2,5000,5000,4599,1,5000.0,17.97,180.69,59280.0,10.51,0.0,...,36 months,D,6 years,MORTGAGE,Source Verified,Current,n,debt_consolidation,w,Individual
3,4000,4000,5468,1,4000.0,18.94,146.51,92000.0,16.74,0.0,...,36 months,D,10+ years,MORTGAGE,Source Verified,Current,n,debt_consolidation,w,Individual
4,30000,30000,829,1,30000.0,16.14,731.78,57250.0,26.35,0.0,...,60 months,C,10+ years,MORTGAGE,Not Verified,Current,n,debt_consolidation,w,Individual
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1652446,15000,15000,6988,1,15000.0,16.02,364.94,85000.0,14.66,0.0,...,60 months,C,< 1 year,RENT,Verified,Fully Paid,n,debt_consolidation,w,Individual
1652447,30000,30000,19206,1,30000.0,30.75,984.47,100000.0,22.13,0.0,...,60 months,F,6 years,MORTGAGE,Verified,Current,n,debt_consolidation,f,Individual
1652448,12000,12000,9592,1,12000.0,14.08,279.72,58000.0,20.88,0.0,...,60 months,C,10+ years,MORTGAGE,Not Verified,Current,n,debt_consolidation,w,Individual
1652449,12000,12000,3497,1,12000.0,25.82,358.01,30000.0,19.28,3.0,...,60 months,E,< 1 year,MORTGAGE,Not Verified,Fully Paid,n,debt_consolidation,w,Joint App


In [4]:
data["grade"].replace({"A":0, "B":1, "C":2, "D":3, "E":4, "F":5, "G":6 },inplace=True)

In [5]:
data = data.drop(['verification_status', 'emp_length', 'dti', 'term', 'open_acc', 'pub_rec', 'loan_status', 'pymnt_plan', 'purpose', 'initial_list_status','application_type', 'policy_code', 'funded_amnt', 'home_ownership', 'funded_amnt_inv', 'revol_bal'], axis = 1)

In [17]:
# data = data.sample(n=10000)

,loan_amnt,int_rate,installment,annual_inc,delinq_2yrs,inq_last_6mths,revol_util,total_acc,out_prncp,out_prncp_inv,total_pymnt,total_pymnt_inv,grade
725640,4000,6.89,123.31,30000.0,0.0,0.0,20.4,7.0,0.00,0.00,4437.522845,4437.52,0
894368,16800,7.26,520.74,100000.0,1.0,0.0,48.2,18.0,0.00,0.00,17896.190000,17896.19,0
350967,25000,7.97,783.07,48000.0,0.0,0.0,41.2,14.0,16651.06,16651.06,10152.240000,10152.24,0
744728,11400,12.69,257.58,50000.0,0.0,0.0,30.1,23.0,0.00,0.00,12749.750000,12749.75,2
258966,16000,16.01,389.18,37440.0,0.0,1.0,50.4,12.0,14133.53,14133.53,3877.570000,3877.57,2
612007,4450,12.74,149.39,36000.0,0.0,0.0,93.5,11.0,1674.53,1674.53,3579.060000,3579.06,2
834641,5200,12.29,173.44,65000.0,0.0,1.0,41.3,13.0,0.00,0.00,5863.490000,5863.49,2
579518,29000,10.78,946.41,75000.0,0.0,2.0,14.2,10.0,0.00,0.00,29183.540000,29183.54,1
177210,19000,16.14,463.46,60000.0,0.0,0.0,65.0,32.0,17484.57,17484.57,3210.150000,3210.15,2
781638,10000,10.99,327.34,38000.0,0.0,0.0,77.1,28.0,0.00,0.00,11765.704747,11765.70,1


In [34]:
data_sample

,loan_amnt,int_rate,installment,annual_inc,delinq_2yrs,inq_last_6mths,revol_util,total_acc,out_prncp,out_prncp_inv,total_pymnt,total_pymnt_inv,grade
1306411,12000,11.55,396.00,40000.0,0.0,0.0,65.3,6.0,0.00,0.00,13805.559998,13805.56,1
1227136,8400,11.49,276.96,49000.0,0.0,2.0,92.8,7.0,3852.81,3852.81,5842.010000,5842.01,1
833010,18000,13.99,418.74,95600.0,12.0,2.0,4.8,43.0,6433.57,6433.57,17991.830000,17991.83,2
19944,30000,8.81,951.35,115000.0,1.0,0.0,38.5,35.0,28532.43,28532.43,1888.020000,1888.02,0
1257105,6625,19.22,243.59,27000.0,0.0,0.0,86.3,10.0,0.00,0.00,8784.312085,8784.31,3


In [7]:
grade = data['grade']
features = data.drop(['grade'], axis =1 )

In [8]:
X_train, X_test, y_train, y_test = train_test_split(features, grade, test_size=0.3, random_state=1)
X_train, X_test, y_train, y_test

(         loan_amnt  int_rate  installment  annual_inc  delinq_2yrs  \
 844441       32000     16.55      1133.74     79000.0          0.0   
 360808       20000     15.05       476.33     46000.0          0.0   
 1440612      24000     15.61       578.68     53000.0          1.0   
 1274733       8000     18.55       291.44     28000.0          0.0   
 1286386      16800     10.99       549.94    101000.0          0.0   
 ...            ...       ...          ...         ...          ...   
 836489        4900     14.65       169.03     32000.0          0.0   
 491263        8000     16.29       282.41    240000.0          2.0   
 470924        6000      7.39       186.34     35000.0          0.0   
 491755        8000      9.16       255.00     89000.0          0.0   
 128037       18000     16.14       634.08     65000.0          0.0   
 
          inq_last_6mths  revol_util  total_acc  out_prncp  out_prncp_inv  \
 844441              0.0        83.0       24.0       0.00           

In [9]:
# Create Decision Tree classifer object
clf = DecisionTreeClassifier()

# Train Decision Tree Classifer
clf = clf.fit(X_train,y_train)

#Predict the response for test dataset
y_pred = clf.predict(X_test)

In [10]:
# Model Accuracy, how often is the classifier correct?
print("Accuracy:",metrics.accuracy_score(y_test, y_pred))

Accuracy: 0.959518776122775


In [11]:
# # Create Decision Tree classifer object
# clf2 = DecisionTreeClassifier(criterion="entropy", max_depth=3)

# # Train Decision Tree Classifer
# clf2 = clf2.fit(X_train,y_train)

# #Predict the response for test dataset
# y_pred = clf2.predict(X_test)

# # Model Accuracy, how often is the classifier correct?
# print("Accuracy:",metrics.accuracy_score(y_test, y_pred))

## results: Accuracy: 0.8575209385640744

In [12]:
# Dump the trained decision tree classifier with Pickle
decision_tree_pkl_filename = 'decision_tree_classifier.pkl'
# Open the file to save as pkl file
decision_tree_model_pkl = open(decision_tree_pkl_filename, 'wb')
pickle.dump(clf, decision_tree_model_pkl)
# Close the pickle instances
decision_tree_model_pkl.close()